In [1]:
import itertools
import functools
import operator
import re
import xarray
import numpy as np
from dataclasses import dataclass

import typing

# Day 1

### Part 1

In [61]:
with open('Advent_2020_Day_1.txt') as f:
    nums = f.read().splitlines()
nums = [int(x) for x in nums]

In [62]:
@dataclass
class Coord:
    a: int
    b: int

In [63]:
pt = Coord(0,1)

while pt.b <= len(nums)-1 and pt.a <= len(nums)-1:
    f,s = int(nums[pt.a]), int(nums[pt.b])
    if f+s == 2020:
        print(pt.a,pt.b)
        print("Sum:",f+s)
        print("Product:", f*s)
        break
        
    #Check if a and b are the same, else add 1
    if pt.a == pt.b:
        pt.b += 1
        # print('1 added to b')
        continue

    #Check if b is at the end, else add 1 to a and reset b
    if pt.b == len(nums) -1:
        pt.a+=1
        pt.b = pt.a + 1
        # print('1 added to a')
        # print('b reset')
        continue
    #Else, add one to b
    pt.b += 1
    # print('1 added to b')
    continue

34 159
Sum: 2020
Product: 1018944


### Part 2: Itertools for conv

In [69]:
comb = list(itertools.combinations(nums,3))
res = [i for i in comb if sum(i) == 2020][0]

In [74]:
functools.reduce(operator.mul,res)

8446464

# Day 2

### Part 1

In [76]:
with open("./Advent_2020_Day_2.txt") as f:
    data = f.read().splitlines()

In [135]:
class instruct:
    def __init__(self,instruction: str):
        
        pattern = "(\d+)-(\d+) (\w): (\w+$)"
        m = re.match(pattern,instruction)
        
        self.min = int(m.group(1))
        self.max = int(m.group(2))
        self.char = m.group(3)
        self.pw = m.group(4)
        
    def validate_pw(self):
        count = self.pw.count(self.char)
        return (count>=self.min and count<=self.max)
    
    def validate_pw_new(self):
        pair = str(self.pw[self.min-1]) + str(self.pw[self.max-1])
        return pair.count(self.char) == 1

In [133]:
valid = sum([instruct(i).validate_pw() for i in data])
valid

524

### Part 2

In [137]:
valid = sum([instruct(i).validate_pw_new() for i in data])
valid

485

# Day 3

### Part 1

In [98]:
with open("./Advent_2020_Day_3.txt") as f:
    data = f.read().splitlines()

In [105]:
#Parse map into array with coords
dmap = list(map(list,data))
dmap = xarray.DataArray(dmap, dims = ["row","col"])

In [179]:
class Explorer:
    def __init__(self,row: int,col: int, dmap: xarray.DataArray):
        self.row = row
        self.col = col
        self.dmap = dmap
        self.treecount = 0
        
    def move_single(self,direction: str, distance: int):
        if direction == "r":
            self.col += 1
        elif direction == "l":
            self.col -= 1
        elif direction == "u":
            self.row += -1
        elif direction == "d":
            self.row += 1
        else:
            raise Exception("Direction unknown")
            
    def validate_position_in_map(self):
        """Checks if current position is still in map
        """
        rowcheck = self.row in dmap.get_index("row")
        colcheck = self.col in dmap.get_index("col")
        
        if rowcheck and colcheck:
            return True
        else:
            return False
    
    def fix_position(self):
        """Fixes position relative to map, since the map can be stacked right-ward.
        """
        if self.col >= dmap.get_index("col").max():
            self.col -= (dmap.get_index("col").max() + 1)
        elif self.row not in dmap.get_index("row"):
            pass
            # raise Exception("Can't be fixed: dropped off row axis")
            
        
    def move_multiple(self,pattern):
        """Moves according to specified pattern, and records if tree is at the end
        """
        
        #Check if still in map, and fix if needed
        if self.validate_position_in_map() == False:
            self.fix_position()       
            
        for i in pattern:
            #Move           
            self.move_single(i,1)
        
        #Check again if still in map, and fix if needed
        if self.validate_position_in_map() == False:
            self.fix_position()  
            
        #Look for tree
        self.record_tree()
        
    def record_tree(self):
        if self.validate_position_in_map() == True:
            if dmap[self.row,self.col] == "#":
                self.treecount += 1

In [180]:
dude = Explorer(0,0,dmap)

while dude.row <= dmap.get_index("row").max():
    dude.move_multiple("rrrd")
dude.treecount

234

### Part 2

In [181]:
digs = []
for pattern in ["rd","rrrd","rrrrrd","rrrrrrrd","rdd"]:
    dude = Explorer(0,0,dmap)
    while dude.row <= dmap.get_index("row").max():
        dude.move_multiple(pattern)
    digs.append(dude.treecount)

In [183]:
functools.reduce(operator.mul, digs)

5813773056

# Day 4

### Part 1

In [8]:
with open("./Advent_2020_Day_4.txt") as f:
    data = f.read().splitlines()

In [151]:
#Parse data
full_data = []
tracker = []
for e in data:
    if e == "":
        full_data.append(tracker)
        tracker = []
        pass
    else:
        tracker.append(e)

#Add last entry
full_data.append(tracker)

In [188]:
class Validator():
    """Validator to check if values in passport fields are valid
    """
    def __init__(self):
        pass
    def validate_key(self,field_key: str, val: str):
        if field_key == 'byr':
            return int(val) <= 2002 and int(val) >= 1920
        elif field_key == 'iyr':
            return int(val) <= 2020 and int(val) >= 2010
        elif field_key == 'eyr': 
            return int(val) <= 2030 and int(val) >= 2020
        elif field_key == 'hgt':
            try:
                num = int(val[:-2])
            except:
                pass
            if val[-2:] == "cm":
                return int(num) <= 193 and int(num) >= 150
            elif val[-2:] == "in":
                return int(num) <= 76 and int(num) >= 59
            else:
                return False
        elif field_key == 'hcl':
            pattern = r"^#([a-f]|[0-9]){6}$"
            return bool(re.match(pattern,val))
        elif field_key == 'ecl':
            return len(val) == 3 and val in set(['amb','blu','brn','gry','grn','hzl','oth'])
        elif field_key == 'pid':
            pattern = r"^\d{9}$"
            return bool(re.match(pattern,val))
        elif field_key == 'cid':
            return True
        else:
            raise Exception("Unknown passport field")
    

In [153]:
class Constants(typing.NamedTuple):
    """Namedtuple to contain constants used
    """
    REQUIRED_FIELDS = ("byr","iyr","eyr","hgt","hcl","ecl","pid")

In [165]:
class PassportChecker():
        
    def __init__(self,passports: list, validator: Validator):
        self.passports = passports
        self.CONSTANTS = Constants()
        self.Validator = Validator
    def split_lists(self):
        """Flattens each list into individual dictionaries of "key:value". 
        """
        
        #Flatten nested lists in the passports
        flattened = []
        for i_list in full_data:
            tracker = []
            for elem in i_list:
                tracker+=(elem.split(" "))
            flattened.append(tracker)
        
        dicts = []
        
        #Convert from individual strings of "key:value" to a dict containing all kv pairs
        for kvpair_list in flattened:
            container = {}
            for kvpair_str in kvpair_list:
                k,v = kvpair_str.split(":")
                container[k] = v
            dicts.append(container)
        
        self.passports = dicts
        
        return flattened
    def validate_passport(self, passport: dict):
        
        return set(self.CONSTANTS.REQUIRED_FIELDS) <= passport.keys()

    def validate_all_passports(self):
        """ Checks if fields are present and adds a valid_passports attribute
        """
        self.valid_passports = sum([self.validate_passport(passport) for passport in self.passports])
    
    def validate_passport_part2(self, passport: dict):
        checks = []
        for k,v in passport.items():
            val = v
            checks.append(Validator.validate_key(self,k,v))
        return all(checks) and self.validate_passport(passport)
    
    def validate_all_passports_part2(self):
        """ Checks if fields are present AND validates the values in each field. 
        Adds a valid_passports_part2 attribute
        """
        self.valid_passports_part2 = sum([self.validate_passport_part2(passport) for passport in self.passports])

In [166]:
a = PassportChecker(full_data, Validator())
a.split_lists()
a.validate_all_passports()
a.valid_passports

256

### Part 2

In [191]:
a.validate_all_passports_part2()
a.valid_passports_part2

198

# Day 5